# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [4]:
%matplotlib widget

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [7]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [8]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [9]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201130_084039_bsize256_test_with_invtransform-scale0.5


In [10]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [11]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [12]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [13]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [14]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,13.0201,1.025850,1.127100,4.94131,11.8516,11.17370,0.807457,0.113676,4.429140,11.6468,502.223,5.81214,"0.092401s mean, 4.34694s max, 0.0267275s min,...","0.059264s mean, 0.524439s max, 0.0193376s min..."
1,10.2727,0.464634,0.409233,3.58681,11.6240,8.52607,1.059390,0.360659,1.323810,11.5644,479.782,5.20281,"0.0881013s mean, 3.71645s max, 0.0244311s min...","0.0530444s mean, 0.42755s max, 0.0152026s min..."
2,10.4935,0.373443,0.340006,4.01128,11.5376,13.35540,0.334809,0.116805,7.178660,11.4502,492.170,6.98537,"0.0874105s mean, 0.819299s max, 0.0249389s mi...","0.0712355s mean, 1.52146s max, 0.0147362s min..."
3,10.5918,0.315664,0.308532,4.22806,11.4791,8.19550,0.090569,0.585326,1.770140,11.4989,523.252,4.96934,"0.0864599s mean, 1.39745s max, 0.0243243s min...","0.050664s mean, 0.466483s max, 0.0154339s min..."
4,10.7141,0.247299,0.240840,4.48438,11.4833,7.40631,0.066236,0.823234,0.794065,11.4456,550.804,5.97390,"0.0873149s mean, 0.863062s max, 0.0248563s mi...","0.060915s mean, 1.35199s max, 0.014805s min, ..."


In [15]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))



interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [47]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']



In [17]:
print(df.shape)
df.tail()

(1319, 21)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec2,chi_spec3,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
1314,14,13140,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.049527,0.038310,0.008020,0.095857,0.037481,4.852472,...,21.722671,28.588184,1,0,"[0.9975028063754142, 1.0981441157090728, 0.311...","[0.004062447530719573, 0.003477212606580231, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[83182.94386431202, 56284.22821412904, 40276.4...","[38732.52845270224, 19203.24919178174, 11861.8...",14-13140
1315,14,13150,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.004531,0.008046,0.004231,0.016808,0.000944,5.210342,...,7.993169,26.033589,1,0,"[1.099090266082445, 0.9684550181048184, 0.2744...","[0.003961204790873815, 0.003408018420842921, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[93935.90086218629, 65773.55935572424, 48213.8...","[48694.811406226, 22908.964219560316, 16542.88...",14-13150
1316,14,13160,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.007626,0.029537,0.006559,0.043722,0.004142,4.774238,...,48.826363,27.429482,1,0,"[1.1605113774669547, 0.9205422490114104, 0.243...","[0.0039098316042676775, 0.0035882118497337994,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[81648.69428792037, 55640.08992702853, 42236.6...","[45765.7014330891, 19304.005971948332, 13939.8...",14-13160
1317,14,13170,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.001344,0.009384,0.004336,0.015064,0.000251,5.294135,...,9.433926,24.680407,1,0,"[1.1190387262955857, 0.9508136600001178, 0.264...","[0.004056786577908398, 0.003328406710984512, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[83458.82654205756, 61098.90116323958, 45182.4...","[43459.97821187262, 24757.737122448394, 15483....",14-13170
1318,14,13180,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.008084,0.007306,0.014269,0.029659,0.006601,5.410288,...,72.900621,23.501847,0,0,"[1.1690823763961509, 0.9001496150054453, 0.243...","[0.004145482512358135, 0.003668566082133083, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[90995.69718130119, 63855.98223417032, 46838.2...","[45650.836596346286, 22435.145780896037, 16805...",14-13180


### View best epochs

#### Locations with best chi_sqr

In [18]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
1305,14,13050,0.000044,0.013042,0.020233,0.033319,0.000137,5.139335,0.221998,1.244321e+01
1054,11,10540,0.013850,0.000998,0.103773,0.118621,0.012158,6.519798,0.420350,2.701527e+01
1311,14,13110,0.033038,0.001831,0.001156,0.036025,0.028934,5.616223,1.369718,1.609707e+02
1276,14,12760,0.003017,0.005899,0.004183,0.013099,0.001067,5.751671,0.118272,1.458427e+01
1145,13,11450,0.000151,0.010376,0.005859,0.016386,0.000134,5.365239,0.187622,1.762481e+01
2,0,20,0.406612,0.294280,0.013186,0.714078,0.177861,1.307446,67404.379588,6.451896e+06
1172,13,11720,0.001507,0.015304,0.003825,0.020635,0.000333,5.071542,0.048730,4.126704e+00
1168,13,11680,0.000868,0.018859,0.010204,0.029931,0.000528,5.015155,0.068299,2.304899e+00


In [19]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [20]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [24]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [25]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [37]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [44]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=2,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, 14]
{'chi_1': 0.039948181567919544, 'chi_spec1': 0.2605606293835334}
[0, 14]
{'chi_1': 0.2771536653873312, 'chi_spec1': 4.71290803644455}
[0, 14]
{'chi_1': 0.039948181567919544, 'chi_spec1': 0.2605606293835334}
[10770 10840 11720 12760 13110 13170]


In [35]:
best_step=[11130]
# best_step=np.arange(40130,40135).astype(int)

In [45]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 21)
[(12, 10770), (12, 10840), (13, 11720), (14, 12760), (14, 13110), (14, 13170)]


In [48]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1077,12-10770,0.018935,0.051433,22.686866,0.001406,0.013084,0.004445,0.001053,5.460232,5.125759,12,10770
1084,12-10840,0.021867,0.337466,25.361722,0.011312,0.008090,0.002465,0.009624,5.253382,50.377482,12,10840
1172,13-11720,0.020635,0.048730,25.822968,0.001507,0.015304,0.003825,0.000333,5.071542,4.126704,13,11720
1276,14-12760,0.013099,0.118272,26.560320,0.003017,0.005899,0.004183,0.001067,5.751671,14.584273,14,12760
1311,14-13110,0.036025,1.369718,25.465371,0.033038,0.001831,0.001156,0.028934,5.616223,160.970715,14,13110
1317,14-13170,0.015064,0.091204,24.680407,0.001344,0.009384,0.004336,0.000251,5.294135,9.433926,14,13170


## Plot pixel intensity and spectrum

In [29]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:1000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [39]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [46]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('12-10770', '12-10840', '13-11720', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [176]:
df[(df.step>4400)& (df.step<5350)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [41]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,11130)

In [42]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [180]:
s_val.shape,img_arr.shape

((1000, 128, 128), (256, 128, 128))

In [181]:
# df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]

### Plotting spread of spectrum of input data

In [182]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [53]:
fname='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201130_084039_bsize256_test_with_invtransform-scale0.5/gen_imgs_chkpt/epoch.12.step.11130_20201201_072154/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy'
np.load(fname).shape


(1000, 1, 128, 128)